# Demo 3: Including other molecules in your polymer system
Most often one wants to study not the dynamics of just one polymer, but of its interactions with a surrounding solvent and with other polymers  
This demo shows how to use polymerist tools to build systems with both coexistent polymers and small-molecule solvents

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

from pathlib import Path
from polymerist.genutils.fileutils.pathutils import is_empty, assemble_path


EXAMPLE_DIR = Path('polymer_loading_examples')
assert EXAMPLE_DIR.exists() and not is_empty(EXAMPLE_DIR)

OUTPUT_DIR = Path('scratch') # dummy directory for writing without tampering with example inputs
OUTPUT_DIR.mkdir(exist_ok=True)

## Packing solvents
`polymerist` provides some utilities for simplifying packing of simulation boxes with a small molecule solvent

The solvent can in principle be any Molecule from which you've obtained partial charges for from your method of choice  
The latter condition is important for avoiding expensive reparameterization downstream, which OpenFF will attempt if charges are not provided

Here, we provide you with pre-parameterized TIP3P water to use for packing; other pre-parameterized solvents [are planned](https://github.com/timbernat/polymerist/issues/4) for future releases

In [2]:
from polymerist.mdtools.openfftools.topology import topology_to_sdf, topology_from_sdf, get_largest_offmol

INFO:rdkit:Enabling RDKit 2023.09.6 jupyter extensions
/home/timber/miniconda3/envs/polymerist-dev/lib/python3.11/site-packages/smirnoff99frosst/smirnoff99frosst.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.


### Load pre-prepared polymer (PNIPAAm)

In [3]:
pnipaam_prepared_sdf = assemble_path(EXAMPLE_DIR, 'PNIPAAm', postfix='prepared', extension='sdf')
assert pnipaam_prepared_sdf.exists()
pnipaam_prepared_top = topology_from_sdf(pnipaam_prepared_sdf)

pnipaam_prepared = get_largest_offmol(pnipaam_prepared_top) # a useful trick for working with individual Molecules in a single-mol or solvated topology
assert pnipaam_prepared.partial_charges is not None

print(pnipaam_prepared.properties)
pnipaam_prepared.visualize(backend='nglview')

{'IUPAC name': 'poly(N-isopropylacrylamide)', 'Common name': 'PNIPAAm', 'First patented': '1956-12-04', 'Patent holder': 'Edward H. Sprecht', 'Patent No.': 'US-2773063-A', 'charge_method': 'NAGL'}


NGLWidget()

### Set periodic box for system

In [4]:
from openmm.unit import nanometer
from openff.toolkit import Molecule
from polymerist.mdtools.openfftools.boxvectors import get_topology_bbox, pad_box_vectors_uniform


box_padding = 0.5 * nanometer      # how far beyond the tight bounding box of the polymer(s) to extend each box face
box_vectors = pad_box_vectors_uniform(
    get_topology_bbox(pnipaam_prepared_top),
    pad_amount=box_padding,
)

### Pack box with solvent of choice to desired density and save solvated Topology

In [5]:
from openmm.unit import gram, centimeter
from polymerist.mdtools.openfftools.solvation.solvents import water_TIP3P
from polymerist.mdtools.openfftools.solvation.packing import pack_topology_with_solvent


rho = 0.997 * gram / centimeter**3 # approximate density of water at 25 C, 1 atm
solvent : Molecule = water_TIP3P   # replace this with your pre-parameterized small molecule of choice, if water is not desired

solvated_polymer_topology = pack_topology_with_solvent(
    pnipaam_prepared_top,
    solvent=solvent,
    box_vecs=box_vectors,
    density=rho,
)
solvated_polymer_topology.visualize()

INFO:polymerist.mdtools.openfftools.solvation.packing:Solvating 58.53939345701369 nm**3 Topology with 1951 water_TIP3P molecules to density of 0.997 g/(cm**3)
INFO:polymerist.mdtools.openfftools.solvation.packing:Packmol packing converged
INFO:polymerist.mdtools.openfftools.solvation.packing:Set solvated Topology box vectors to [[3.3314000129699703 0.0 0.0] [0.0 4.1878999710083 0.0] [0.0 0.0 4.1959000587463375]] nanometer


NGLWidget()

In [6]:
solv_path = assemble_path(OUTPUT_DIR, pnipaam_prepared.name, postfix=f'solv_{solvent.name}', extension='sdf')
topology_to_sdf(solv_path, solvated_polymer_topology)

## Coexistent polymers (polymer melts)
`polymerist` ships with a lattice-based packer which makes it straightforward to "tile" multiple copies of a polymer (or many polymers) into a simulation box  
The packer is flexible and allows you to position polymers in space via a set of "lattice points", which define where to center to coordinates of each copy of a conformer

We demonstrate setting up a loosely-packed alternating lattice of two polymers (PNIPAAm and PBPA) that [I prepared for you earlier](https://tvtropes.org/pmwiki/pmwiki.php/Main/OneIPreparedEarlier)

### Polymer 1: poly(N-isopropylacrylamide) (PNIPAAm)
We already loaded this one for the solvation demo earlier

In [7]:
pnipaam_prepared.visualize(backend='nglview')

NGLWidget()

### Polymer 2 : poly(bisphenol A carbonate) (PBPA)

In [8]:
pbpa_prepared_sdf = assemble_path(EXAMPLE_DIR, 'PBPA', postfix='prepared', extension='sdf')
assert pbpa_prepared_sdf.exists()
pbpa_prepared_top = topology_from_sdf(pbpa_prepared_sdf)

pbpa_prepared = get_largest_offmol(pbpa_prepared_top) # a useful trick for working with individual Molecules in a single-mol or solvated topology
assert pbpa_prepared.partial_charges is not None

print(pbpa_prepared.properties)
pbpa_prepared.visualize(backend='nglview')

{'IUPAC name': 'poly(bisphenol A carbonate)', 'Common name': 'Makrolon', 'First patented': '1953-10-16', 'Patent holder': 'Hermann Schnell, Bayer AG', 'Patent No.': 'US-3028365', 'charge_method': 'NAGL'}


NGLWidget()

### Sizing lattice and calculating site locations

In [9]:
S : int = 3 # number of polymers to place along each axis (i.e., will have SxSxS alternating box of polymers)
polymer_1 : Molecule = pnipaam_prepared
polymer_2 : Molecule = pbpa_prepared

In [10]:
import numpy as np
from itertools import product as cartesian
from polymerist.mdtools.openfftools.physprops import effective_radius


# generate cordinate for lattice; polymers wil be placed concentric to these lattice sites
lattice_str = f'{S}x{S}x{S}'
integer_lattice = np.array([int_point for int_point in cartesian(range(S), repeat=3)]) # the "3" is because we are in 3 dimensions
is_odd_idx = np.mod(integer_lattice.sum(axis=1), 2).astype(bool) # analogous to the indices of either color in a 3D checkboard

r_eff = max(effective_radius(polymer_1), effective_radius(polymer_2))   # scale by larger of effective radii to avoid collisions;
lattice_points = (r_eff * integer_lattice).m_as('angstrom')             # strip units while ensuring magnitudes are as Angstroms

### Packing each polymer onto alternating lattice sites

In [11]:
from polymerist.mdtools.openfftools.topology import topology_from_molecule_onto_lattice


pnipaam_top_packed   = topology_from_molecule_onto_lattice(polymer_1, lattice_points[~is_odd_idx])
bisphenol_top_packed = topology_from_molecule_onto_lattice(polymer_2, lattice_points[is_odd_idx])
mixed_polymer_top = pnipaam_top_packed + bisphenol_top_packed
mixed_polymer_top.visualize()

NGLWidget()

In [15]:
melt_name = f'{polymer_1.name}_{polymer_2.name}_{lattice_str}'
melt_sdf_path = assemble_path(OUTPUT_DIR, f'{melt_name}_melt', 'sdf')
topology_to_sdf(melt_sdf_path, mixed_polymer_top) # export the mixed polymer topology to SDF

## Solvated polymer melts
In fact, there's no reason we can't _also_ solvate a packed melt using the methods we've already shown!

In [16]:
box_padding = 0.5 * nanometer # typical nonbonded cutoffs are around 0.9 nm, so this 0.5 on either side of the box ensures our polymers don't self-interact
rho = 0.2 * gram / centimeter**3 # low density here is to account for wide spacing of lattice melt, and so this demo doesn't take forever to finish :P
solvent : Molecule = water_TIP3P

box_vectors = pad_box_vectors_uniform(get_topology_bbox(mixed_polymer_top), box_padding)
solvated_melt_topology = pack_topology_with_solvent(
    mixed_polymer_top,
    solvent=solvent,
    box_vecs=box_vectors,
    density=rho,
)
solvated_melt_topology.visualize()

INFO:polymerist.mdtools.openfftools.solvation.packing:Solvating 5075.43250307511 nm**3 Topology with 33933 water_TIP3P molecules to density of 0.2 g/(cm**3)


INFO:polymerist.mdtools.openfftools.solvation.packing:Packmol packing converged
INFO:polymerist.mdtools.openfftools.solvation.packing:Set solvated Topology box vectors to [[17.286203772801315 0.0 0.0] [0.0 16.25189441312063 0.0] [0.0 0.0 18.066309093960562]] nanometer


NGLWidget()

In [17]:
solv_path = assemble_path(OUTPUT_DIR, melt_name, postfix=f'solv_{solvent.name}', extension='sdf')
topology_to_sdf(solv_path, solvated_melt_topology)

## Looks pretty good, right?
In the [next set of demos](../3.0-index.ipynb), we'll show how to export these types of system to MD inputs, and how to run simulations in series with OpenMM